http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [26]:
import numpy as np
np.random.seed(32)
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [4]:
# class RNN:
#     def step(self, x):
#         #update the hidden state
#         self.h = np.tanh()

data = open('input.txt', 'r').read()
# print data


In [69]:

# rand = np.random.RandomState(12)
np.random.seed(32)

hidden_size = 100
seq_length = 25
p = 0
# data = "hello"
data = open('input.txt', 'r').read()
chars = list(set(data))
vocab_size = len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
inputs  = [char_to_ix[ch] for ch in data[p     : p + seq_length ]]
targets = [char_to_ix[ch] for ch in data[p + 1 : p + seq_length + 1]]
Wxh = np.random.randn(hidden_size, vocab_size)  * 0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)  * 0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size , 1)) # output bias
hprev = np.zeros((hidden_size,1))

learning_rate = 1e-1
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)

def lossFun(inputs, targets, hprev):
# def lossFun(inputs, hprev):
    """
    inputs, targes are both list of integers.
    hprev is Hx1 array of initial state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
#     hs[-1] = np.copy(hprev)
    loss = 0
    hs[-1] = np.copy(hprev)
    
    # forward pass
    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
#         print 'xs[t]', xs[t]
        dot_x = np.dot(Wxh, xs[t])
        dot_h_prev = np.dot(Whh, hs[t-1])

#         print 'dot_x', dot_x
#         print 'dot_h_prev', dot_h_prev

        
        hs[t] = np.tanh(dot_x + dot_h_prev + bh) # hidden state
#         print 'hs[t]', hs[t]
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilites for next chars
#         print 'ys[i]', ys[t]
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        l = -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
#         print 'l', l
        loss += l
#     print 'loss', loss
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
#     print 'ps', ps
    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
#         print 'dy', dy
        dy[targets[t]] -= 1  # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
#         print 'dy', dy
        dWhy += np.dot(dy, hs[t].T)
#         print 'dWhy', dWhy
        dby += dy
#         print 'dby', dby
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
#         print 'dh', dh
    
#         print 'dh', dh
#         print 'hs[t][0]', hs[t][0]
#         print 'dh[0]', dh[0]
#         print 'hs[t]', hs[t]
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
#         print 'dhraw[0]', dhraw[0]
        dbh += dhraw
#         print 'dbh', dbh
    
#         print 'dbh', dbh
    
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
#         print 'dhnext', dhnext

#         print 'dh', dh
#         print 'dbh', dbh
#         print 'dWxh', dWxh
#         print 'dWhy', dWhy
#         print 'dWhh', dWhh
#         print 'dby', dby

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
#     print 'inputs', h, seed_ix, n
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        if t == 0:
#             print 'wxh', Wxh
#             print 'x', x
#             print 'whh', Whh
#             print 'intputs[0]', seed_ix
            print 'h', h
#             print 'bh', bh
#             print 'h1', Wxh,
#             print 'Wxh', Wxh
#             print 'h2', Whh, h
#             print 'h3', bh#             print 'y', y
        p = np.exp(y) / np.sum(np.exp(y))
#         if t == 0:
#             print 'p', p
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


losses = []
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
for i in range(5):

    if p+seq_length+1 >= len(data) or i == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
#     print 'inputs', inputs
#     print 'targets', targets
#     print 'wxh', Wxh
#     print 'whh', Whh
#     print 'why', Why
#     print 'bh', bh
#     print 'by', by
#     print 'h_prev', hprev

    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
#     print 'hprev', hprev
#     print 'dwxh', dWxh
#     print 'dwhh', dWhh
#     print 'dwhy', dWhy
#     print 'dbh', dbh
#     print 'dby', dby
    
#     print 'dWxh', dWxh
    
    p += seq_length # move data pointer

    mWxh += dWxh * dWxh
    mWhh += dWhh * dWhh
    mWhy += dWhy * dWhy
    mbh += dbh * dbh
    mby += dby * dby
    

    Wxh += -learning_rate * dWxh / np.sqrt(mWxh + 1e-8) # adagrad update
    Whh += -learning_rate * dWhh / np.sqrt(mWhh + 1e-8)
    Why += -learning_rate * dWhy / np.sqrt(mWhy + 1e-8)
    bh += -learning_rate * dbh / np.sqrt(mbh + 1e-8)
    by += -learning_rate * dby / np.sqrt(mby + 1e-8)

#     print 'wxh', Wxh
#     print 'whh', Whh
#     print 'why', Why
#     print 'bh',  bh
#     print 'by',  by
    
#     print 'Wxh', Wxh

    if i % 100 == 0:
#         print inputs
        sample_ix = sample(hprev, inputs[0], 200)
#         txt = ''.join(ix_to_char[ix] for ix in sample_ix)
#         print '-----\n %s \n------' % (txt,)
#         print 'loss', loss
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
#     if i % 100 == 0: print 'iter %d, loss: %f' % (i, smooth_loss) # print progress
    losses.append(loss)

#     print loss

# plt.plot(losses)
# print losses

h [[ 0.01832831]
 [-0.21719961]
 [-0.20568907]
 [-0.24236387]
 [-0.2103193 ]
 [ 0.02150432]
 [ 0.20759876]
 [ 0.20007644]
 [ 0.20830389]
 [ 0.1856988 ]
 [ 0.00040154]
 [-0.00924468]
 [-0.20757687]
 [ 0.20603051]
 [ 0.01420357]
 [ 0.01270558]
 [ 0.00276013]
 [ 0.18224357]
 [-0.23414466]
 [ 0.24801997]
 [-0.23151675]
 [ 0.04903263]
 [-0.02593918]
 [-0.02388683]
 [ 0.03467857]
 [ 0.18871837]
 [ 0.00998784]
 [ 0.00917616]
 [-0.18681041]
 [ 0.18424572]
 [-0.18661908]
 [ 0.00981638]
 [-0.19455784]
 [ 0.24511228]
 [-0.22763786]
 [-0.22558603]
 [ 0.22301616]
 [-0.0029261 ]
 [-0.23983623]
 [-0.0176378 ]
 [ 0.21519047]
 [-0.18712575]
 [ 0.24918568]
 [-0.23525583]
 [-0.01829251]
 [ 0.00239445]
 [-0.01790335]
 [-0.20723261]
 [-0.01542303]
 [-0.00520361]
 [ 0.00319696]
 [-0.19667616]
 [ 0.00816098]
 [-0.02410593]
 [-0.19025361]
 [-0.24347447]
 [-0.00850076]
 [-0.22230596]
 [-0.00774856]
 [-0.03219417]
 [ 0.2097315 ]
 [ 0.02498915]
 [ 0.04847822]
 [ 0.18244793]
 [ 0.21764299]
 [ 0.18221728]
 [-0.002

In [74]:
p = 0
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
hprev = np.zeros((hidden_size,1))
print inputs
print targets
# print hprev
loss, a, _, _, _, _, _ = lossFun(inputs, targets, hprev)
# print loss
print a

[31, 81, 65, 15, 40, 0, 31, 40, 15, 15, 40, 13, 83, 44, 9, 82, 0, 81, 66, 66, 82, 64, 17, 86, 65]
[81, 65, 15, 40, 0, 31, 40, 15, 15, 40, 13, 83, 44, 9, 82, 0, 81, 66, 66, 82, 64, 17, 86, 65, 82]
[[-0.01287879  0.          0.         ...,  0.          0.         -0.01581844]
 [-0.00474426  0.          0.         ...,  0.          0.          0.00425313]
 [ 0.0178115   0.          0.         ...,  0.          0.          0.00580286]
 ..., 
 [ 0.00044909  0.          0.         ...,  0.          0.          0.00384821]
 [-0.01153658  0.          0.         ...,  0.          0.         -0.003235  ]
 [-0.00535685  0.          0.         ...,  0.          0.          0.00869725]]
